Необходимо понять какую метрику использовать чтобы оценивать качество моделей.

Я брал за основу следующие работы:

https://arxiv.org/pdf/1312.6229.pdf - OverFeat - это статья группы ЛеКуна которая предлагает общий фреймворк для классификации/детектирования объектов на изображении

https://arxiv.org/pdf/1503.01640.pdf - 

http://dircweb.kingston.ac.uk/papers/Yin,%20Fei2007_841079/PerformanceEvaluationofObjectTrackingAlgorithms.pdf - 

https://www.merl.com/publications/docs/TR2006-041.pdf


